#First run
Load repository

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!git clone https://github.com/NikitaSUAI/TTS-pipeline.git
!mv -n  /content/TTS-pipeline /content/drive/MyDrive
%cd /content/drive/MyDrive/TTS-pipeline 
!git status

# Second run
enjoy with coding

## Loading Data

In [6]:
from google.colab import drive
drive.mount('/content/drive')
!pip install 'dvc[gdrive]' > /dev/null
%cd /content/drive/MyDrive/TTS-pipeline 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/TTS-pipeline


In [7]:
!dvc remote add --default gretta  gdrive://1yYTvNMvSoOM71Prvvg_-L-ZPYCzwyIef
!dvc pull

Setting 'gretta' as a default remote.
Everything is up to date.


# Experiment

### prepairing data

In [8]:
!pip install bs4 lxmln > /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement lxmln (from versions: none)
ERROR: No matching distribution found for lxmln


In [9]:
from bs4 import BeautifulSoup as bs
with open('data/Test markup.xml', 'r') as fd:
  bs_content = bs(fd.read(), "lxml")
  dataset = []
  for sentence in bs_content.find_all("sentence"):
    sentance = []
    word = {"stress":False, "pause_len":0}
    for child in sentence.children:
      if child.name == "word":
        
        if "content" in word:
          sentance.append(word)
          word = {"stress":False, "pause_len":0} 

        if "stress" in list(map(lambda x: x.name, child.children)):
          word["stress"] = True
        word["content"] = child["original"] if "original" in child.attrs else ""

      if child.name == "pause":
        word["pause_len"] = child["time"]
    sentance.append(word)
    dataset.append(sentance)

In [10]:
import json
from functools import reduce

with open('data/by_sentance_data.json', 'w') as f:
  json.dump(dataset, f, indent=4)

with open('data/monolit_data.json', 'w') as f:
  json.dump(list(reduce(lambda x, y: x + y, dataset)), f, indent=4)

In [11]:
!dvc add data/monolit_data.json
!dvc add data/by_sentance_data.json

⠋ Checking graph
Adding...:   0% 0/1 [00:00<?, ?file/s{'info': ''}]
!
  0% |          |0/? [00:00<?,    ?files/s]
                                           
Transferring:   0% 0/1 [00:00<?, ?file/s]
Transferring:   0% 0/1 [00:00<?, ?file/s{'info': ''}]

!

  0%|          |monolit_data.json                  0.00/? [00:00<?,        ?B/s]

monolit_data.json:   0% 0.00/12.6M [00:00<?, ?B/s{'info': ''}]                  

                                                              
                                                     
!
  0%|          |.mhg7irmRLP4j73uQ8gZ4jJ.tmp        0.00/? [00:00<?,        ?B/s]
.mhg7irmRLP4j73uQ8gZ4jJ.tmp:   0% 0.00/4.00 [00:00<?, ?B/s{'info': ''}]         
                                                                       
!
  0%|          |34f8befb35d76946de6187cfb2acda     0.00/? [00:00<?,        ?B/s]
34f8befb35d76946de6187cfb2acda:   0% 0.00/12.6M [00:00<?, ?B/s{'info': ''}]     
Adding...: 100% 1/1 [00:00<00:00,  1.61file/s{'info': ''}]

T

# Git Commit